---
title:  Tensorflow2 模型保存和重载 save and restore
tags: 小书匠,tensorflow2,keras,save,restore,reload,load,h5,checkpoint,ckpt,saved_model,save_weights
grammar_cjkRuby: true
renderNumberedHeading: true
---

[toc]

# Tensorflow2 模型保存和重载 save and restore

In [65]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.4.1


### 获取示例数据集

要演示如何保存和加载权重，您将使用 [MNIST 数据集](http://yann.lecun.com/exdb/mnist/). 要加快运行速度，请使用前1000个示例：

In [51]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### 定义模型

首先构建一个简单的序列（sequential）模型：

In [52]:
# 定义一个简单的序列模型

def create_model():
    model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    return model

# 创建一个基本的模型实例
model = create_model()

# 显示模型的结构
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_21 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 在训练期间保存模型（以 checkpoints 形式保存）

您可以使用训练好的模型而无需从头开始重新训练，或在您打断的地方开始训练，以防止训练过程没有保存。 `tf.keras.callbacks.ModelCheckpoint` 允许在训练的**过程中**和**结束时**回调保存的模型。

### Checkpoint callback 

创建一个只在训练期间保存权重的 `tf.keras.callbacks.ModelCheckpoint` 回调：

In [66]:
checkpoint_path = "training/model.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个保存模型权重的回调
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True, # 这个一定要有
                                                 verbose=1)

# 使用新的回调训练模型
model.fit(train_images, 
          train_labels,  
          epochs=2,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 通过回调训练

Epoch 1/2
32/32 [==============================] - 0s 5ms/step - loss: 0.0517 - accuracy: 0.9930 - val_loss: 0.4294 - val_accuracy: 0.8700

Epoch 00001: saving model to training/model.ckpt
Epoch 2/2
32/32 [==============================] - 0s 3ms/step - loss: 0.0407 - accuracy: 0.9990 - val_loss: 0.4000 - val_accuracy: 0.8760

Epoch 00002: saving model to training/model.ckpt


这将创建一个 TensorFlow checkpoint 文件集合，这些文件在每个 epoch 结束时更新：

In [67]:
!ls {checkpoint_dir}

checkpoint                     cp.index
cp.ckpt.data-00000-of-00001    model.ckpt.data-00000-of-00001
cp.ckpt.index                  model.ckpt.index
cp.data-00000-of-00001


In [68]:
# 创建一个基本模型实例
model = create_model()

# 评估模型
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.4005 - accuracy: 0.1000
Untrained model, accuracy: 10.00%


然后从 checkpoint 加载权重并重新评估：

In [69]:
# 加载权重
model.load_weights(checkpoint_path)

# 重新评估模型
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4000 - accuracy: 0.8760
Restored model, accuracy: 87.60%


### checkpoint 回调选项

#### save_weights_only

- False（default)，根据文件名保存成 h5 格式或者 saved_model 格式(使用 `model.save(filepath)`)
    - 如果文件名以 .h5 结尾，保存成 h5 格式
    - 否则保存成 saved_model 格式
- True，那么保存成 checkpoint 格式(使用 `model.save_weights(filepath)`)

#### 将 epoch 保存在文件名中

In [58]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个回调，每 5 个 epochs 保存模型的权重
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True)

# 创建一个新的模型实例
model = create_model()

# 使用新的回调训练模型
model.fit(train_images,  train_labels,
          epochs=6, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00006: saving model to training_2/cp-0006.ckpt


由于我们将 epoch 保存在了文件名中，我们在读取文件的时候通常会读取那个最新的 checkpoint，可以使用

In [59]:
latest_checkpoint = tf.train.latest_checkpoint(os.path.dirname(checkpoint_path))
model = create_model()
model.load_weights(latest_checkpoint)
print("Restore from {}".format(latest_checkpoint))

Restore from training_2/cp-0006.ckpt


#### period 指定多少个 step 保存一次

In [60]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个回调，每 5 个 epochs 保存模型的权重
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=2)

# 创建一个新的模型实例
model = create_model()

# 使用新的回调训练模型
model.fit(train_images, 
          train_labels,
          epochs=6, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00006: saving model to training_2/cp-0006.ckpt


#### 保存 h5 文件

`tf.keras.callbacks.ModelCheckpoint` 既可以保存 checkpoint 格式，又可以保存 h5 格式、saved_model 格式。主要是通过传入的文件名来区分的，如果文件名后缀是 .h5 ，那么会保存成 h5 文件。否则，会保存成 checkpoint 文件。

In [74]:
checkpoint_path = "model.h5"

# 创建一个保存模型权重的回调
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 使用新的回调训练模型
model.fit(train_images, 
          train_labels,  
          epochs=2,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 通过回调训练

Epoch 1/2
32/32 [==============================] - 0s 6ms/step - loss: 0.0192 - accuracy: 1.0000 - val_loss: 0.4133 - val_accuracy: 0.8770

Epoch 00001: saving model to model.h5
Epoch 2/2
32/32 [==============================] - 0s 3ms/step - loss: 0.0162 - accuracy: 1.0000 - val_loss: 0.4200 - val_accuracy: 0.8730

Epoch 00002: saving model to model.h5


In [75]:
!ls {checkpoint_path}

model.h5


## 手动保存权重

您将了解如何将权重加载到模型中。使用 `Model.save_weights` 方法手动保存它们同样简单。默认情况下， `tf.keras` 和 `save_weights` 特别使用 TensorFlow [checkpoints](../../guide/keras/checkpoints) 格式 `.ckpt` 扩展名和 ( 保存在 [HDF5](https://js.tensorflow.org/tutorials/import-keras.html) 扩展名为 `.h5`  [保存并序列化模型](../../guide/keras/save_and_serialize#weights-only_saving_in_savedmodel_format) )：

In [63]:
# 保存权重
model.save_weights('./checkpoints/my_checkpoint')

# 创建模型实例
model = create_model()

# 恢复权重
model.load_weights('./checkpoints/my_checkpoint')

# 评估模型
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4423 - accuracy: 0.8530
Restored model, accuracy: 85.30%


## 保存整个模型

调用 [`model.save`](https://tensorflow.google.cn/api_docs/python/tf/keras/Model#save) 将保存模型的结构，权重和训练配置保存在单个文件/文件夹中。这可以让您导出模型，以便在不访问原始 Python 代码*的情况下使用它。因为优化器状态（optimizer-state）已经恢复，您可以从中断的位置恢复训练。

整个模型可以以两种不同的文件格式（`SavedModel` 和 `HDF5`）进行保存。需要注意的是 TensorFlow 的 `SavedModel` 格式是 TF2.x. 中的默认文件格式。但是，模型仍可以以  `HDF5` 格式保存。下面介绍了以两种文件格式保存整个模型的更多详细信息。

保存完整模型会非常有用——您可以在 TensorFlow.js（[Saved Model](https://tensorflow.google.cn/js/tutorials/conversion/import_saved_model), [HDF5](https://tensorflow.google.cn/js/tutorials/conversion/import_keras)）加载它们，然后在 web 浏览器中训练和运行它们，或者使用  TensorFlow Lite 将它们转换为在移动设备上运行（[Saved Model](https://tensorflow.google.cn/lite/convert/python_api#converting_a_savedmodel_), [HDF5](https://tensorflow.google.cn/lite/convert/python_api#converting_a_keras_model_)）

\*自定义对象（例如，子类化模型或层）在保存和加载时需要特别注意。请参阅下面的**保存自定义对象**部分

### SavedModel 格式

SavedModel 格式是序列化模型的另一种方法。以这种格式保存的模型，可以使用 `tf.keras.models.load_model` 还原，并且模型与 TensorFlow Serving 兼容。[SavedModel 指南](https://tensorflow.google.cn/guide/saved_model)详细介绍了如何提供/检查 SavedModel。以下部分说明了保存和还原模型的步骤。

In [13]:
# 创建并训练一个新的模型实例。
model = create_model()
model.fit(train_images, train_labels, epochs=2)

# 将整个模型另存为 SavedModel。
!mkdir -p saved_model
model.save('saved_model/my_model')

Train on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 1s 517us/sample - loss: 1.1599 - accuracy: 0.6770
Epoch 2/2
1000/1000 [==============================] - 0s 89us/sample - loss: 0.4437 - accuracy: 0.8770
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


SavedModel 格式是一个包含 protobuf 二进制文件和 Tensorflow 检查点（checkpoint）的目录。检查保存的模型目录：

In [64]:
# my_model 文件夹
!ls saved_model

# 包含一个 assets 文件夹，saved_model.pb，和变量文件夹。
!ls saved_model/my_model

my_model
assets         saved_model.pb variables


从保存的模型重新加载新的 Keras 模型：

In [15]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# 检查其架构
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


还原的模型使用与原始模型相同的参数进行编译。 尝试使用加载的模型运行评估和预测：

### HDF5 格式

Keras使用 [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) 标准提供了一种基本的保存格式。

In [18]:
# 创建并训练一个新的模型实例
model = create_model()
model.fit(train_images, train_labels, epochs=2)

# 将整个模型保存为 HDF5 文件。
# '.h5' 扩展名指示应将模型保存到 HDF5。
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 1s 528us/sample - loss: 1.1697 - accuracy: 0.6710
Epoch 2/2
1000/1000 [==============================] - 0s 94us/sample - loss: 0.4292 - accuracy: 0.8830


现在，从该文件重新创建模型：

In [19]:
# 重新创建完全相同的模型，包括其权重和优化程序
new_model = tf.keras.models.load_model('my_model.h5')

# 显示网络结构
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Keras 通过检查网络结构来保存模型。这项技术可以保存一切:

* 权重值
* 模型的架构
* 模型的训练配置(您传递给编译的内容)
* 优化器及其状态（如果有的话）（这使您可以在中断的地方重新开始训练）

Keras 无法保存 `v1.x` 优化器（来自 `tf.compat.v1.train`），因为它们与检查点不兼容。对于 v1.x 优化器，您需要在加载-失去优化器的状态后，重新编译模型。


### 保存自定义对象

如果使用的是 SavedModel 格式，则可以跳过此部分。HDF5 和 SavedModel 之间的主要区别在于，HDF5 使用对象配置保存模型结构，而 SavedModel 保存执行图。因此，SavedModel 能够保存自定义对象，例如子类化模型和自定义层，而无需原始代码。

要将自定义对象保存到 HDF5，必须执行以下操作:

1. 在对象中定义一个 `get_config` 方法，以及可选的 `from_config` 类方法。
  * `get_config(self)` 返回重新创建对象所需的参数的 JSON 可序列化字典。
  * `from_config(cls, config)` 使用从 get_config 返回的 config 来创建一个新对象。默认情况下，此函数将使用 config 作为初始化 kwargs（`return cls(**config)`）。
2. 加载模型时，将对象传递给 `custom_objects` 参数。参数必须是将字符串类名称映射到 Python 类的字典。例如，`tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`

有关自定义对象和 `get_config` 的示例，请参见[从头开始编写层和模型](https://tensorflow.google.cn/guide/keras/custom_layers_and_models)教程。


# References
- http://localhost:8888/lab/tree/DL-Project/learnTensorflow/Tensorflow2%20guide/save_and_load.ipynb
- [保存和恢复模型  |  TensorFlow Core](https://www.tensorflow.org/tutorials/keras/save_and_load)
- [tf.keras.callbacks.ModelCheckpoint  |  TensorFlow Core v2.4.1](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint)